### install required libraries 

In [30]:
!pip install -q  google-generativeai python-dotenv

### Load Environment variables 

In [31]:
import os 
from dotenv import load_dotenv
load_dotenv()
gemini_key = os.environ.get('GEMINI_API_KEY')
openweather_key = os.environ.get('OPENWEATHERMAP_API_KEY')
if gemini_key and openweather_key:
    print('API keys fetched successfully')

API keys fetched successfully


### import necessary modules 

In [32]:
import google.generativeai as genai
import re 
import os 
import requests
from datetime import datetime,timedelta

### Creating the Agent Class

In [33]:


class ChatHistoryManager:
    def __init__(self, filename="travel_agent_chat_history.txt", max_file_size_mb=5):
        self.history = []
        self.filename = filename
        self.max_file_size_mb = max_file_size_mb

    def add_message(self, role, text):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.history.append(
            {'role': role, 'text': text, 'timestamp': timestamp})

    def save_to_file(self):
        self._rotate_file_if_needed()
        with open(self.filename, "a", encoding="utf-8") as file:
            for message in self.history:
                file.write(
                    f"{message['timestamp']} {message['role']}: {message['text']}\n")
        self.history.clear()

    def display(self):
        for message in self.history:
            print(
                f"{message['timestamp']} {message['role']}: {message['text']}")

    def _rotate_file_if_needed(self):
        if not os.path.exists(self.filename):
            with open(self.filename, "a", encoding="utf-8") as file:
                pass

        if os.path.getsize(self.filename) > self.max_file_size_mb * 1024 * 1024:
            os.rename(self.filename, self.filename + ".backup")


class TravelAgent:
    def __init__(self, system_prompt, initial_budget=1000):
     

        # Configure Generative AI
        genai.configure(api_key=gemini_key)

        self.system_prompt = system_prompt
        self.budget = initial_budget
        self.history_manager = ChatHistoryManager()
        self.history_manager.add_message("system", system_prompt)
        self.history_manager.add_message("system", f"The current budget is ${self.budget}.")
        
        self.generation_config = {
            "temperature": 0.7,
            "top_p": 1,
            "top_k": 1,
            "max_output_tokens": 2048,
        }

        self.safety_settings = {
            "HARM_CATEGORY_HARASSMENT": "BLOCK_NONE",
            "HARM_CATEGORY_HATE_SPEECH": "BLOCK_NONE",
            "HARM_CATEGORY_SEXUALLY_EXPLICIT": "BLOCK_NONE",
            "HARM_CATEGORY_DANGEROUS_CONTENT": "BLOCK_NONE",
        }

        self.model = genai.GenerativeModel(
            'gemini-pro', generation_config=self.generation_config, safety_settings=self.safety_settings
        )
        self.chat = self.model.start_chat(history=[])

    def __call__(self, user_input):
        if user_input.lower() == "history":
            self.history_manager.display()
            return

        if user_input.lower() == "restart":
            self.history_manager.save_to_file()
            os.system('cls' if os.name == 'nt' else 'clear')
            self.history_manager.add_message("system", "--- New Session ---")
            self.chat = self.model.start_chat(history=[])
            return "Session restarted."

        if user_input.lower() == "exit":
            self.history_manager.save_to_file()
            return "Session ended. Chat history saved."

        try:
            # Append user input to history and send to model
            self.history_manager.add_message("user", user_input)
            response = self.chat.send_message(user_input, stream=True)
            
            response_text = ""
            for chunk in response:
                if chunk.text.endswith("."):
                    response_text += chunk.text
                else:
                    response_text += re.sub(r'\s*$', '.', chunk.text)
                print(chunk.text, end="", flush=True)  # Stream response to user

            # Save response to history
            self.history_manager.add_message("assistant", response_text)
            return response_text

        except Exception as e:
            return f"An error occurred: {e}"

    def set_budget(self, amount):
        self.budget = amount
        self.history_manager.add_message("system", f"The current budget is now ${self.budget}.")

    def get_budget(self):
        return self.budget


## Tools for the Agent 

### Currency convertor 


In [34]:
def convert_currency(amount,from_currency,to_currency):
    import requests
    url = "https://api.exchangerate-api.com/v4/latest/" + from_currency
    response = requests.get(url)
    data = response.json()
    rate = data['rates'][to_currency]
    converted_amount = amount * rate
    return round(converted_amount,2)


In [35]:
convert_currency(100,'USD','INR')

8497.0

### Weather Tool

In [36]:
def get_weather(location:str,days_ahead:int =0)->str:
    import datetime
    api_key = os.environ.get("OPENWEATHERMAP_API_KEY") 
    future_date = datetime.date.today() + datetime.timedelta(days=days_ahead)
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={location}&appid={api_key}&units=metric&dt={int(datetime.datetime(future_date.year,future_date.month,future_date.day,12,0,0).timestamp())}"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200:
        try:
            temperature = data['list'][0]['main']['temp']
            description = data['list'][0]['weather'][0]['description']
            country = data['city']['country'] #Added this line to extract country
            return f"The weather in {location}, {country} on {future_date} will be {description} with a temperature of {temperature}°C."
        except (KeyError, IndexError):
            return f"Error getting weather data for {location} on {future_date}.  Data not found in response."

    else:
        return f"Error getting weather data for {location} on {future_date}. Status code: {response.status_code}"


In [37]:
get_weather('hyderabad',3)

'Error getting weather data for hyderabad on 2024-12-21. Status code: 401'

### Translation tool

In [38]:
from deep_translator import GoogleTranslator

def translate_text(text, target_language):
    """
    Translate text into the specified target language using deep-translator.

    Parameters:
        text (str): The text to translate.
        target_language (str): The target language code (e.g., 'en' for English, 'fr' for French).

    Returns:
        str: The translated text.
    """
    try:
        # Initialize the translator
        translator = GoogleTranslator(source='auto', target=target_language)
        
        # Perform translation
        translated_text = translator.translate(text)
        
        return translated_text
    except Exception as e:
        return f"An error occurred: {e}"

In [39]:
# Example usage:
original_text = "Hola, ¿cómo estás?"
target_language = "en"  # Translate to English
translated_text = translate_text(original_text, target_language)
print("Translated text:", translated_text)

Translated text: Hello how are you?


### Prompt 

In [40]:
travel_agent_prompt=""" You are a Travel Planning Assistant. Your role is to help users plan their trips by providing information about destinations, currency conversion, weather forecasts, and language translation.

You run in a loop of Thought, Action, PAUSE, Observation.
Use Thought to describe your reasoning about the user's request.
Use Action to perform one of the available actions, then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

1. convert_currency:
             e.g., convert_currency: 100, USD, EUR
             Converts the given amount from one currency to another.

2. get_weather:
             e.g. get_weather: Paris, 3
             Gets the weather forecast for the specified city and number of days ahead (e.g. for today).

3. translate_text:
             e.g., translate_text: Hello, how are you?, fr
             Translates the given text to the specified target language (use language codes like 'fr' for French).

Always consider the user's budget when making suggestions. The current budget will be provided in the system messages.

Example session:

User: I'm planning a trip to Paris next week. What should I know?

Thought: I should check the weather in Paris for next week and provide some basic French phrases.
Action: get_weather: Paris, 7

PAUSE

Observation: Partly cloudy, 22°C

Thought: Now that I have the weather information, I should provide a simple French phrase and its translation.
Action: translate_text: Hello, how are you?, fr

PAUSE

Observation: Bonjour, comment allez-vous?

Answer: Great! For your trip to Paris next week, here's what you should know:

1. Weather: It's expected to be partly cloudy with a temperature of 22°C (72°F). Pack accordingly with light layers and maybe a light jacket for evenings.

2. Language: While many people in Paris speak some English, it's always appreciated if you try some basic French. Here's a useful phrase: "Hello, how are you?" in French is "Bonjour, comment allez-vous?"

Remember to consider the budget for your trip and let me know if you need any currency conversion or have any other questions about your Paris adventure!

"""

## Buidling  the AI Travel Agent 



### Directory of available actions 


In [41]:
known_actions = {
    "convert_currency": convert_currency,
    "get_weather": get_weather,
    "translate_text": translate_text
}


### AI Travel Agent 

In [42]:
def plan_trip(question,max_turns=1):
    agent =TravelAgent(travel_agent_prompt)

    budget_match=re.search(r'\$(\d+)',question)
    if budget_match:
        budget=int(budget_match.group(1))
        agent.set_budget(budget)

    next_prompt=question
    
    action_re=re.compile(r'^Action:(\w+):(.+)$')

    for i in range(max_turns):
        result=agent(next_prompt)
        print(result)

        actions=[action_re.match(a) for a in result.split('\n') if action_re.match(a)]

        if actions:
            action,action_input=actions[0].groups()

            if action not in known_actions:
                print(f"Unknown action :{action}:{action_input}")
                continue

            print(f'-- running {action}:{action_input}')

            try:
                if action =='convert_currency':
                    amount,from_currency,to_currency=action_input.split(',')
                    observation=known_actions[action](float(amount),from_currency.strip(),to_currency.strip())
                elif action == 'get_weather':
                    location,*days=action_input.split(',')
                    days_ahead=int(days[0]) if days else 0
                    observation=known_actions[action](location.strip(),days_ahead)
                elif action =='translate_text':
                    if ',' in action_input:
                        text,target_lang=action_input.rsplit(',',1)
                    else:
                        text ,target_lang= action_input.rsplit(None,1)

                    observation = known_actions[action](text.strip(),target_lang.strip())

                print(f'Observation : {observation}')
                agent.add_to_memory(action,observation)
                next_prompt= f'Observation:{observation}'
            except  Exception as e:
                print(f"Error executing action :{e}")
                next_prompt=f'Error : Unable to execute action {action}.please try again'
        else:
            return
                         
                                                      

### Experiment


In [44]:
question = "i am planing a 3 days  trip to vizag in the next week . my budget is $2000.what  should i prepare ?"
result=plan_trip(question)

**Preparation for a 3-Day Trip to Visakhapatnam (Vizag) with a Budget of $2000**

**Accommodation (approx. $200):**

* Consider booking a comfortable hotel or guesthouse near the beach or the city center.
* Look for deals or discounts on online booking websites.

**Transportation (approx. $100):**

* Pre-book a taxi or hire a private driver for airport transfers and local transportation.
* Utilize public transportation (buses, autos) for short distances.

**Activities and Experiences (approx. $500):**

* Visit the picturesque Rushikonda Beach and indulge in water sports like parasailing or jet skiing.
* Explore the historic Borra Caves, known for their stunning limestone formations.
* Take a boat ride to Dolphin's Nose for panoramic views of the coast.
* Visit the INS Kursura Submarine Museum for an interactive naval experience.

**Food and Drinks (approx. $300):**

* Sample the local cuisine at restaurants serving fresh seafood, vegetarian thalis, and traditional Andhra dishes.
* Expe